In [1]:
import pandas as pd
# read one of the filted data sets for useage
data = pd.read_csv('../data/filtered_data/training_data_ros.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12490 entries, 0 to 12489
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   num__avtisst                   12490 non-null  float64
 1   num__dnrday                    12490 non-null  float64
 2   cat_simple__dnr                12490 non-null  float64
 3   num__adlsc                     12490 non-null  float64
 4   num__prg6m                     12490 non-null  float64
 5   num__adls                      12490 non-null  float64
 6   cat_one_hot__dzgroup_COPD      12490 non-null  float64
 7   num__sps                       12490 non-null  float64
 8   num__aps                       12490 non-null  float64
 9   num__hday                      12490 non-null  float64
 10  cat_one_hot__dzclass_ARF/MOSF  12490 non-null  float64
 11  target                         12490 non-null  float64
dtypes: float64(12)
memory usage: 1.1 MB


In [3]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score, classification_report
from scipy.special import expit  # More stable sigmoid function


## Initial version: binomial classification

In [7]:
# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Custom feature transformation
def trans_matrix_for_splines(X):
    # X is (n_samples, 9)
    # Example: add squared terms for nonlinear regression
    X_squared = X ** 2
    intercept = np.ones((X.shape[0], 1))
    return np.hstack([intercept, X, X_squared])

# Logistic loss function (negative log-likelihood)
def average_likelihood_fn(beta, X, y):
    
    # z for linear prediction of logit transformed probability being 1
    z = X @ beta
    
    # p for predicted probability of y being 1
    p = sigmoid(z) # a function of X and beta
    
    # given data and beta, the likelihood of this beta value is 
    epsilon = 1e-9 # to avoid log(0)
    neg_likelihood_beta = -np.mean(y * np.log(p + epsilon) + (1 - y) * np.log(1 - p + epsilon))
    return neg_likelihood_beta # the beta that minimizes this is the best fit on the given data

# Fit model
def fit_logistic(X, y):
    
    # transform/map the design matrix to a higher dimentional space for splines
    X_transformed = trans_matrix_for_splines(X)
    
    # set an initial value to beta vector (0 for now)
    beta_init = np.zeros(X_transformed.shape[1])
    
    # using optimization function from scipy to minimize the negative log-likelihood
    result = minimize(average_likelihood_fn, beta_init, args=(X_transformed, y), method='BFGS')
    
    # return MLE beta (average likelihood function minimized)
    mle_beta = result.x
    return mle_beta

In [ ]:
# test the above functions on a simulation data 
# Simulate data
def simulate_logistic_data(n_samples=500, n_features=9, seed=42):
    np.random.seed(seed)
    
    # Simulate predictors X
    X = np.random.normal(0, 1, size=(n_samples, n_features))
    
    # True beta (including intercept)
    true_beta = np.array([0.5] + [(-1)**i * 0.8 / (i+1) for i in range(n_features * 2)])  # length = 1 + 2*n_features
    
    # Transform X for splines: includes intercept, X, and X^2
    X_transformed = trans_matrix_for_splines(X)
    
    # Compute probabilities using logistic model
    logits = X_transformed @ true_beta
    probs = expit(logits)
    
    # Simulate binary outcome
    y = np.random.binomial(1, probs)
    
    return X, y, true_beta

# --- Example usage ---
X_sim, y_sim, beta_true = simulate_logistic_data()
estimated_beta = fit_logistic(X_sim, y_sim)

print("True beta (first 10):", np.round(beta_true[:10], 4))
print("Estimated beta (first 10):", np.round(estimated_beta[:10], 4))

True beta (first 10): [ 0.5     0.8    -0.4     0.2667 -0.2     0.16   -0.1333  0.1143 -0.1
  0.0889]
Estimated beta (first 10): [ 0.517   1.0153 -0.3586  0.3572 -0.1673  0.0607 -0.0186  0.1453 -0.1212
  0.0313]


### Waiting to be checked below

In [ ]:
# waiting for the one-vs-rest logistic regression function to be implemented
# fit one-vs-rest logistic regression for multiclass classification
def fit_multiclass_logistic_ovr(X, y, num_classes):
    models = {}
    for cls in range(num_classes):
        y_binary = (y == cls).astype(int)  # One-vs-Rest
        beta = fit_logistic(X, y_binary)   # Use your earlier function
        models[cls] = beta
    return models

# Predict for multiclass
def predict_multiclass(X, models):
    X_transformed = trans_matrix_for_splines(X)
    probs = {}
    for cls, beta in models.items():
        probs[cls] = sigmoid(X_transformed @ beta)
    
    # Combine predictions into a matrix (n_samples x num_classes)
    prob_matrix = np.vstack([probs[cls] for cls in sorted(probs.keys())]).T
    return np.argmax(prob_matrix, axis=1)

## Version 2: transform X to a new design matrix for splines

In [ ]:
# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [6]:
# Custom design matrix mapping function
def truncated_power(x, knot, degree):
    return np.where(x > knot, (x - knot) ** degree, 0.0)

def transform_column_with_splines(x_col, degree=3, knots=None, include_intercept=False):
    """
    Transform a single column using spline basis expansion.
    """
    base = [x_col**d for d in range(1, degree + 1)]
    if include_intercept:
        base = [np.ones_like(x_col)] + base
    if knots:
        for knot in knots:
            base.append(truncated_power(x_col, knot, degree))
    return np.column_stack(base)

def transform_matrix_with_splines(X, knots_dict = None, degree=3, include_intercept=False):
    """
    Apply spline transformation to all columns in X with different knots per column.

    Args:
        X: (n, p) input matrix
        knots_dict: dict mapping column index to list of knots for that column
        degree: spline degree (default = 3 for cubic)
        include_intercept: whether to include an intercept term per column (default = False)

    Returns:
        Transformed design matrix (n, ?)
    """
    X = np.asarray(X)
    n, p = X.shape
    transformed_columns = []
    
    # Transform X for splines with transform_matrix_with_splines : includes intercept,
    quantile_knots = [0.25, 0.5, 0.75]    # assuming columns have been standardized

    knots_dict = {
       i: quantile_knots for i in range(X.shape[1])
    }

    for j in range(p):
        x_col = X[:, j]
        knots = knots_dict.get(j, [])  # empty if not specified
        X_j_spline = transform_column_with_splines(x_col, degree, knots, include_intercept)
        transformed_columns.append(X_j_spline)

    return np.hstack(transformed_columns)

In [1]:
# test the above functions on a simulation data 
# Simulate data
def simulate_logistic_data(n_samples=500, n_features=15, seed=42):
    np.random.seed(seed)
    # Simulate predictors X
    X = np.random.normal(0, 1, size=(n_samples, n_features))
    
    # Transform X for splines with transform_matrix_with_splines : includes intercept,
    # quantile_knots = [0.25, 0.5, 0.75]    # assuming columns have been standardized

    # knots_dict = {
    #    i: quantile_knots for i in range(X.shape[1])
    # }
    
    X_transformed = transform_matrix_with_splines(X, degree = 2, include_intercept=True)
    
    transformed_n_features = X_transformed.shape[1]
    print("Transformed shape:", X_transformed.shape)
    
    # True beta (including intercept)
    true_beta = np.array([(-1)**i * 0.8 / (i+1) for i in range(transformed_n_features)])  # length = # of transformed_n_features
    print("True beta shape:", true_beta.shape)
    
    
    # Compute probabilities using logistic model
    logits = X_transformed @ true_beta
    probs = expit(logits)
    
    # Simulate binary outcome
    y = np.random.binomial(1, probs)
    
    return X, y, true_beta

In [9]:
# change the knots to check 
import LogsiticRegre as lr
X_sim, y_sim, beta_true = simulate_logistic_data()
transformed_x_sim = lr.transform_matrix_with_splines(X_sim, degree=3, include_intercept=True)
estimated_beta = lr.fit_logistic(X_sim, y_sim)

print("True beta (first 10):", np.round(beta_true[:10], 4))
print("Estimated beta (first 10):", np.round(estimated_beta[:10], 4))

Transformed shape: (500, 90)
True beta shape: (90,)
True beta (first 10): [ 0.8    -0.4     0.2667 -0.2     0.16   -0.1333  0.1143 -0.1     0.0889
 -0.08  ]
Estimated beta (first 10): [-0.2339 -0.3019 -0.048  -0.1609 -0.0775 -0.0187 -0.1425 -0.0806 -0.128
  0.0913]


## Version 3: Estimate prior for beta vector